原文：http://www.datakit.cn/blog/2017/02/05/t_sne_full.html  
  
<br>  
  
## T-SNE    
T-SNE 是一种非线性的降维方法；适合把数据降维到2维或3维进行可视化  
  
<br>  
  
## SNE   
原理：  

SNE通过仿射变换，把数据点映射到概率分布上：  
1、SNE构建一个高维对象之间的概率分布，使得相似的对象有更高的概率被选择，而不相似的对象有较低的概率被选择  
  
2、SNE在低维空间里在构建这些点的概率分布，使得这两个概率分布之间尽可能的相似。  
  
<br>  

推导：  

1、将欧几里得距离转换为条件概率来表达点与点之间的相似度;概率pij，正比于xi和xj之间的相似度（这种概率是我们自主构建的）有一个参数是σi，对于不同的点xi取值不一样，(一个样本$x_i$,与其他样本的相似度用概率表示，概率越大表示与某个样本越可能相似;其中这里的概率是条件概率) 
$$p_{j|i}=\frac{\frac{exp(-||x_i-x_j||^2)}{2\sigma_i^2}}{\sum_{k \neq i } \frac{exp(-||x_i-x_k||^2)}{2\sigma_i^2}}$$  
  
  
  
2、在低维空间中样本之间的概率分布（在低维中样本映射为y,**且我们可以指定让降维后的样本去服从什么的概率分布**，这里我们指定样本服从方差为$\frac{1}{\sqrt{2}}$的高斯分布）  
$$q_{j|i}=\frac{exp(-||y_i-y_j||^2)}{\sum_{k \neq i } exp(-||y_i-y_k||^2)}$$   
  
 <br> 
  
如果降维的效果比较好，局部特征保留完整，那么 $p_{ij}=q_{ij}$ 那么根据KL散度，最小化C
$$C=\sum_{i} KL(P_i|Q_i)=\sum_{i}\sum_{j}p_{j|i}log \frac{p_{j|i}}{q_{j|i}} $$  
  
<br>  

  
分析：  
在P的概率密度大的地方，它应该尽量和Q概率密度大的区域保持一致以保证KL散度小，而在P概率密度很小的地方，P和Q的差别对KL的影响很小。   
所以t_SNE倾向于保留局部特征。  

<br>  
  
<br>  
  
* 在上面的介绍中，该如何选择每个$x_i$的$\sigma_i$:  

    首先不同的$x_i$有不同的$\sigma_i$，而且随着$\sigma_i$的增大，熵$p_i$的也会增大。  
      
    SNE使用困惑度的概念:  
      
    困惑度：$P_{erp}(p_i)=2^{H(p_i)}\qquad\qquad$ $H(p_i)$是熵：$H(p_i)=-\underset{j}{\sum}p_{j|i}log_2 p_{j|i}$   
        
    为一个$x_i$设置一个$\sigma_i$，之后就得到这个$x_i$的困惑度：$P_{erp}(p_i)$:  
      
    直观来说，如果概率分布的熵较大，那么其分布的形状就相对平坦，该分布中每个元素的概率就更相近一些,而我们的困惑度会随着熵增加  
      
    而变大，因此如果我们希望有更高的困惑度，那么所有的 $p_{j|i}$（对于给定的 i）就会彼此更相近一些 换而言之，如果我们希望概率分  
    
    布 $P_i$ 更加平坦，那么我们就可以增大$σ_i$。我们配置的$σ_i$ 越大，概率分布中所有元素的出现概率就越接近于 1/N。实际上增大  
    
    $σ_i$  会增加每个点的近邻数，这就是为什么我们常将困惑度参数大致等同于所需要的近邻数量  
      
        
          
<br>  

<br>  

在确定KL散度公式中$\sigma_i$之后，现在开始求这个公式的最小值：  
  
 $\frac{\partial C}{\partial y}=2\underset{j}{\sum}(p_{j|i}-q_{j|i}+p_{i|j}-q_{i|j} )(y_i-y_j)$   
   
 $Y^t=Y^{t-1} +\eta \frac{\partial C}{\partial Y} +\alpha(t)(Y^{t-1}-Y^{t-2})$   
 
    
<br> 
  
  
     
SNE存在的问题：难优化    
  
    
<br>  
  
<br>  
  
<br>  

## T-SNE  
与SNE的不同：  
1、使用对称版的SNE，简化梯度公式  
  
2、低维空间下，使用t分布替代高斯分布表达两点之间的相似度    
  
    
<br>  
  
对称SNE,概率不再是条件概率(使用联合概率分布来替换条件概率分布），而是联合概率 $p_{ij}$： 
  
  $p_{ij}=\frac{\frac{exp(-||x_i-x_j||^2)}{2\sigma^2}}{\sum_{k \neq l } \frac{exp(-||x_k-x_l||^2)}{2\sigma^2}}$   
    
  <br>
    
  在低维不用高斯分布，而是用t分布（自由度为1）：$q_{ij}=\frac{exp(-||y_i-y_j||^2)^{-1}}{\sum_{k \neq l } exp(-||y_k-y_l||^2)^{-1}}$  
    
  <br>  
    
  $C=\sum_{i} KL(P_i|Q_i)=\sum_{i}\sum_{j}p_{ij}log \frac{p_{ij}}{q_{ij}} $  
    
  <br>     
       
  $\frac{\partial C}{\partial y}=4\underset{j}{\sum}(p_{ij}-q_{ij} )(y_i-y_j)(1-||y_i-y_j||^2)^{-1}$      
